# <span style="color:#4e5ec5">Import Packages</span>


In [156]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
from datetime import datetime
import calendar
from math import sin, cos, sqrt, atan2, radians,asin
import folium
from folium import FeatureGroup, LayerControl, Map, Marker
from folium.plugins import HeatMap
from folium.plugins import TimestampedGeoJson
from folium.plugins import MarkerCluster
from geopy.distance import great_circle
import matplotlib.dates as mdates
import matplotlib as mpl
from datetime import timedelta
import datetime as dt
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', -1)
plt.style.use('fivethirtyeight')
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import pickle
from geopy.distance import geodesic
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgbm
import xgboost as xgb 

%matplotlib inline




# <span style="color:#4e5ec5">Load Train and Test Datasets</span>


In [157]:
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
train.head()

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,label
0,189123628,10.5,834,56,0.0000,64,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,270.32,correct
1,189125358,10.5,791,47,0.0000,134,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,197.85,correct
2,189125719,10.5,1087,80,0.0000,61,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,301.64,correct
3,189127273,10.5,598,271,15.6638,68,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,82.30,correct
4,189129552,10.5,3407,182,0.0000,112,11/1/2019 5:38,11/1/2019 6:35,7.13402,79.8969,6.91865,79.8649,1065.02,correct


In [158]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16967 entries, 0 to 16966
Data columns (total 14 columns):
tripid                       16967 non-null int64
additional_fare              16967 non-null float64
duration                     16967 non-null int64
meter_waiting                16967 non-null int64
meter_waiting_fare           16967 non-null float64
meter_waiting_till_pickup    16967 non-null int64
pickup_time                  16967 non-null object
drop_time                    16967 non-null object
pick_lat                     16967 non-null float64
pick_lon                     16967 non-null float64
drop_lat                     16967 non-null float64
drop_lon                     16967 non-null float64
fare                         16967 non-null float64
label                        16967 non-null object
dtypes: float64(7), int64(4), object(3)
memory usage: 1.8+ MB


# <span style="color:#4e5ec5">Pre process pickup and drop off times</span>

### <span style="color:#4ec57a"> Remove outliers</span>

In [159]:
train = train.loc[train['duration'] > 0]

In [160]:
train = train.loc[train['fare'] > 0]

In [161]:
train = train.loc[train['fare'] < 11000]

In [162]:
train = train.loc[train['meter_waiting_till_pickup'] < 10000]
train = train.loc[train['meter_waiting_fare'] < 10000]

### <span style="color:#4ec57a"> Convert pickup and drop off time columns to date type. Split the Pickup time and Drop time in to year-month-day-day of week-hour</span>

In [163]:
train['pickup_time']=pd.to_datetime(train['pickup_time'])
train['drop_time']=pd.to_datetime(train['drop_time'])
train.head()

test['pickup_time']=pd.to_datetime(test['pickup_time'])
test['drop_time']=pd.to_datetime(test['drop_time'])

In [164]:
# beak the pickup date time
train['pickup_date']= train['pickup_time'].dt.date
train['pickup_day']=train['pickup_time'].apply(lambda x:x.day)
train['pickup_hour']=train['pickup_time'].apply(lambda x:x.hour)
train['pickup_day_of_week']=train['pickup_time'].apply(lambda x:x.dayofweek)
train['pickup_month']=train['pickup_time'].apply(lambda x:x.month)
train['pickup_year']=train['pickup_time'].apply(lambda x:x.year)

test['pickup_date']= test['pickup_time'].dt.date
test['pickup_day']=test['pickup_time'].apply(lambda x:x.day)
test['pickup_hour']=test['pickup_time'].apply(lambda x:x.hour)
test['pickup_day_of_week']=test['pickup_time'].apply(lambda x:x.dayofweek)
test['pickup_month']=test['pickup_time'].apply(lambda x:x.month)
test['pickup_year']=test['pickup_time'].apply(lambda x:x.year)

# beak the drop off date time
train['drop_date']= train['drop_time'].dt.date
train['drop_day']=train['drop_time'].apply(lambda x:x.day)
train['drop_hour']=train['drop_time'].apply(lambda x:x.hour)
train['drop_day_of_week']=train['drop_time'].apply(lambda x:x.dayofweek)
train['drop_month']=train['drop_time'].apply(lambda x:x.month)
train['drop_year']=train['drop_time'].apply(lambda x:x.year)

test['drop_date']= test['drop_time'].dt.date
test['drop_day']=test['drop_time'].apply(lambda x:x.day)
test['drop_hour']=test['drop_time'].apply(lambda x:x.hour)
test['drop_day_of_week']=test['drop_time'].apply(lambda x:x.dayofweek)
test['drop_month']=test['drop_time'].apply(lambda x:x.month)
test['drop_year']=test['drop_time'].apply(lambda x:x.year)

train.head()

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,...,pickup_hour,pickup_day_of_week,pickup_month,pickup_year,drop_date,drop_day,drop_hour,drop_day_of_week,drop_month,drop_year
0,189123628,10.5,834,56,0.0000,64,2019-11-01 00:20:00,2019-11-01 00:34:00,6.86252,79.8993,...,0,4,11,2019,2019-11-01,1,0,4,11,2019
1,189125358,10.5,791,47,0.0000,134,2019-11-01 00:56:00,2019-11-01 01:09:00,6.88589,79.8984,...,0,4,11,2019,2019-11-01,1,1,4,11,2019
2,189125719,10.5,1087,80,0.0000,61,2019-11-01 01:08:00,2019-11-01 01:26:00,6.90839,79.8651,...,1,4,11,2019,2019-11-01,1,1,4,11,2019
3,189127273,10.5,598,271,15.6638,68,2019-11-01 02:27:00,2019-11-01 02:37:00,6.92570,79.8895,...,2,4,11,2019,2019-11-01,1,2,4,11,2019
4,189129552,10.5,3407,182,0.0000,112,2019-11-01 05:38:00,2019-11-01 06:35:00,7.13402,79.8969,...,5,4,11,2019,2019-11-01,1,6,4,11,2019


### <span style="color:#4ec57a">New column to hold label as boolean value</span>

In [165]:
train['label_bool']=train['label'].apply(lambda x:x=='correct')
train.head()

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,...,pickup_day_of_week,pickup_month,pickup_year,drop_date,drop_day,drop_hour,drop_day_of_week,drop_month,drop_year,label_bool
0,189123628,10.5,834,56,0.0000,64,2019-11-01 00:20:00,2019-11-01 00:34:00,6.86252,79.8993,...,4,11,2019,2019-11-01,1,0,4,11,2019,True
1,189125358,10.5,791,47,0.0000,134,2019-11-01 00:56:00,2019-11-01 01:09:00,6.88589,79.8984,...,4,11,2019,2019-11-01,1,1,4,11,2019,True
2,189125719,10.5,1087,80,0.0000,61,2019-11-01 01:08:00,2019-11-01 01:26:00,6.90839,79.8651,...,4,11,2019,2019-11-01,1,1,4,11,2019,True
3,189127273,10.5,598,271,15.6638,68,2019-11-01 02:27:00,2019-11-01 02:37:00,6.92570,79.8895,...,4,11,2019,2019-11-01,1,2,4,11,2019,True
4,189129552,10.5,3407,182,0.0000,112,2019-11-01 05:38:00,2019-11-01 06:35:00,7.13402,79.8969,...,4,11,2019,2019-11-01,1,6,4,11,2019,True


### check for missing values

In [166]:
train[pd.isnull(train)].sum()
test[pd.isnull(test)].sum()

tripid                       0.0
additional_fare              0.0
duration                     0.0
meter_waiting                0.0
meter_waiting_fare           0.0
meter_waiting_till_pickup    0.0
pickup_time                  0.0
drop_time                    0.0
pick_lat                     0.0
pick_lon                     0.0
drop_lat                     0.0
drop_lon                     0.0
fare                         0.0
pickup_date                  0.0
pickup_day                   0.0
pickup_hour                  0.0
pickup_day_of_week           0.0
pickup_month                 0.0
pickup_year                  0.0
drop_date                    0.0
drop_day                     0.0
drop_hour                    0.0
drop_day_of_week             0.0
drop_month                   0.0
drop_year                    0.0
dtype: float64

In [167]:
train.describe()

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pick_lat,pick_lon,drop_lat,drop_lon,...,pickup_day,pickup_hour,pickup_day_of_week,pickup_month,pickup_year,drop_day,drop_hour,drop_day_of_week,drop_month,drop_year
count,1.694100e+04,16941.000000,16941.000000,16941.000000,16941.000000,16941.000000,16941.000000,16941.000000,16941.000000,16941.000000,...,16941.000000,16941.000000,16941.000000,16941.000000,16941.000000,16941.000000,16941.00000,16941.000000,16941.000000,16941.000000
mean,2.014766e+08,12.980599,1550.978809,466.825748,22.762010,107.988726,6.965297,79.998102,6.966021,79.999279,...,15.860221,13.807154,2.919603,7.948527,2019.340004,15.861165,14.08624,2.921846,7.947288,2019.340122
std,7.451844e+06,13.071511,3632.783957,3237.889130,187.216382,157.720781,0.210795,0.242942,0.211416,0.243116,...,8.847364,4.817967,1.940625,5.003889,0.473724,8.847079,4.88101,1.941447,5.004307,0.473764
min,1.891236e+08,5.000000,22.000000,0.000000,0.000000,0.000000,5.937600,79.819100,5.941860,79.796700,...,1.000000,0.000000,0.000000,1.000000,2019.000000,1.000000,0.00000,0.000000,1.000000,2019.000000
25%,1.947495e+08,10.500000,548.000000,33.000000,0.000000,26.000000,6.864870,79.870400,6.865580,79.870100,...,8.000000,10.000000,1.000000,1.000000,2019.000000,8.000000,11.00000,1.000000,1.000000,2019.000000
50%,2.014010e+08,10.500000,1009.000000,123.000000,0.116600,73.000000,6.914020,79.898600,6.916010,79.899100,...,16.000000,14.000000,3.000000,11.000000,2019.000000,16.000000,14.00000,3.000000,11.000000,2019.000000
75%,2.083329e+08,10.500000,1796.000000,351.000000,14.305617,144.000000,7.043040,79.956500,7.046890,79.960100,...,23.000000,18.000000,5.000000,12.000000,2020.000000,23.000000,18.00000,5.000000,12.000000,2020.000000
max,2.138173e+08,450.820000,162747.000000,162277.000000,9460.749100,8800.000000,8.051010,81.009600,8.064930,81.009700,...,31.000000,23.000000,6.000000,12.000000,2020.000000,31.000000,23.00000,6.000000,12.000000,2020.000000


### <span style="color:#4ec57a">Function created to calculate distance based on longitudes and latitudes of pickup and drop off locations</span>

In [168]:
def haversine_distance(lat1, long1, lat2, long2):
    data = [train, test]
    for i in data:
        R = 6371  #radius of earth in kilometers
        #R = 3959 #radius of earth in miles
        phi1 = np.radians(i[lat1])
        phi2 = np.radians(i[lat2])
    
        delta_phi = np.radians(i[lat2]-i[lat1])
        delta_lambda = np.radians(i[long2]-i[long1])
    
        #a = sin²((φB - φA)/2) + cos φA . cos φB . sin²((λB - λA)/2)
        a = np.sin(delta_phi / 2.0) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2.0) ** 2
    
        #c = 2 * atan2( √a, √(1−a) )
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    
        #d = R*c
        d = (R * c) #in kilometers
        i['H_Distance'] = d
    return d

In [169]:
haversine_distance('pick_lat', 'pick_lon', 'drop_lat', 'drop_lon')
train['H_Distance'].head(10)

0    5.092770 
1    3.168058 
2    6.305395 
3    0.861946 
4    24.207039
5    4.777624 
6    5.322544 
7    1.035302 
8    2.930715 
9    14.381001
Name: H_Distance, dtype: float64

In [170]:
train.describe()

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pick_lat,pick_lon,drop_lat,drop_lon,...,pickup_hour,pickup_day_of_week,pickup_month,pickup_year,drop_day,drop_hour,drop_day_of_week,drop_month,drop_year,H_Distance
count,1.694100e+04,16941.000000,16941.000000,16941.000000,16941.000000,16941.000000,16941.000000,16941.000000,16941.000000,16941.000000,...,16941.000000,16941.000000,16941.000000,16941.000000,16941.000000,16941.00000,16941.000000,16941.000000,16941.000000,16941.000000
mean,2.014766e+08,12.980599,1550.978809,466.825748,22.762010,107.988726,6.965297,79.998102,6.966021,79.999279,...,13.807154,2.919603,7.948527,2019.340004,15.861165,14.08624,2.921846,7.947288,2019.340122,4.518685
std,7.451844e+06,13.071511,3632.783957,3237.889130,187.216382,157.720781,0.210795,0.242942,0.211416,0.243116,...,4.817967,1.940625,5.003889,0.473724,8.847079,4.88101,1.941447,5.004307,0.473764,4.580300
min,1.891236e+08,5.000000,22.000000,0.000000,0.000000,0.000000,5.937600,79.819100,5.941860,79.796700,...,0.000000,0.000000,1.000000,2019.000000,1.000000,0.00000,0.000000,1.000000,2019.000000,0.000000
25%,1.947495e+08,10.500000,548.000000,33.000000,0.000000,26.000000,6.864870,79.870400,6.865580,79.870100,...,10.000000,1.000000,1.000000,2019.000000,8.000000,11.00000,1.000000,1.000000,2019.000000,1.485219
50%,2.014010e+08,10.500000,1009.000000,123.000000,0.116600,73.000000,6.914020,79.898600,6.916010,79.899100,...,14.000000,3.000000,11.000000,2019.000000,16.000000,14.00000,3.000000,11.000000,2019.000000,3.214649
75%,2.083329e+08,10.500000,1796.000000,351.000000,14.305617,144.000000,7.043040,79.956500,7.046890,79.960100,...,18.000000,5.000000,12.000000,2020.000000,23.000000,18.00000,5.000000,12.000000,2020.000000,6.144517
max,2.138173e+08,450.820000,162747.000000,162277.000000,9460.749100,8800.000000,8.051010,81.009600,8.064930,81.009700,...,23.000000,6.000000,12.000000,2020.000000,31.000000,23.00000,6.000000,12.000000,2020.000000,79.220774


In [171]:
def late_night (row):
    if (row['pickup_hour'] <= 6) or (row['pickup_hour'] >= 22):
        return 1
    else:
        return 0


def rush_hours (row, column):
    if ((row[column] <= 20) and (row[column] >= 18)) and (row['pickup_day_of_week'] < 5):
        return 1
    else:
        return 0

train['rush_hours']=train.apply (lambda x: rush_hours(x, 'pickup_hour'), axis=1)
test['rush_hours']= test.apply (lambda x: rush_hours(x, 'pickup_hour'), axis=1)

train['late_night']=train.apply (lambda x: late_night(x), axis=1)
test['late_night']= test.apply (lambda x: late_night(x), axis=1)

In [172]:
train.head(50)

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,...,pickup_month,pickup_year,drop_date,drop_day,drop_hour,drop_day_of_week,drop_month,drop_year,label_bool,H_Distance
0,189123628,10.5,834,56,0.000000,64,2019-11-01 00:20:00,2019-11-01 00:34:00,6.86252,79.8993,...,11,2019,2019-11-01,1,0,4,11,2019,True,5.092770
1,189125358,10.5,791,47,0.000000,134,2019-11-01 00:56:00,2019-11-01 01:09:00,6.88589,79.8984,...,11,2019,2019-11-01,1,1,4,11,2019,True,3.168058
2,189125719,10.5,1087,80,0.000000,61,2019-11-01 01:08:00,2019-11-01 01:26:00,6.90839,79.8651,...,11,2019,2019-11-01,1,1,4,11,2019,True,6.305395
3,189127273,10.5,598,271,15.663800,68,2019-11-01 02:27:00,2019-11-01 02:37:00,6.92570,79.8895,...,11,2019,2019-11-01,1,2,4,11,2019,True,0.861946
4,189129552,10.5,3407,182,0.000000,112,2019-11-01 05:38:00,2019-11-01 06:35:00,7.13402,79.8969,...,11,2019,2019-11-01,1,6,4,11,2019,True,24.207039
5,189132829,10.5,1246,487,0.000000,133,2019-11-01 06:29:00,2019-11-01 06:49:00,6.84371,79.9051,...,11,2019,2019-11-01,1,6,4,11,2019,True,4.777624
6,189135103,10.5,1333,295,17.198500,212,2019-11-01 06:50:00,2019-11-01 07:12:00,6.90760,79.9524,...,11,2019,2019-11-01,1,7,4,11,2019,True,5.322544
7,189139296,10.5,360,80,4.664000,3,2019-11-01 07:00:00,2019-11-01 07:06:00,7.26706,80.6064,...,11,2019,2019-11-01,1,7,4,11,2019,True,1.035302
8,189138671,10.5,1539,588,33.986400,43,2019-11-01 07:02:00,2019-11-01 07:28:00,6.85137,79.9537,...,11,2019,2019-11-01,1,7,4,11,2019,True,2.930715
9,189135814,10.5,26,19,0.000000,73,2019-11-01 07:27:00,2019-11-01 07:27:00,7.00355,79.8819,...,11,2019,2019-11-01,1,7,4,11,2019,False,14.381001


In [173]:
train.columns

Index(['tripid', 'additional_fare', 'duration', 'meter_waiting',
       'meter_waiting_fare', 'meter_waiting_till_pickup', 'pickup_time',
       'drop_time', 'pick_lat', 'pick_lon', 'drop_lat', 'drop_lon', 'fare',
       'label', 'pickup_date', 'pickup_day', 'pickup_hour',
       'pickup_day_of_week', 'pickup_month', 'pickup_year', 'drop_date',
       'drop_day', 'drop_hour', 'drop_day_of_week', 'drop_month', 'drop_year',
       'label_bool', 'H_Distance'],
      dtype='object')

In [174]:
test.columns

Index(['tripid', 'additional_fare', 'duration', 'meter_waiting',
       'meter_waiting_fare', 'meter_waiting_till_pickup', 'pickup_time',
       'drop_time', 'pick_lat', 'pick_lon', 'drop_lat', 'drop_lon', 'fare',
       'pickup_date', 'pickup_day', 'pickup_hour', 'pickup_day_of_week',
       'pickup_month', 'pickup_year', 'drop_date', 'drop_day', 'drop_hour',
       'drop_day_of_week', 'drop_month', 'drop_year', 'H_Distance'],
      dtype='object')

In [175]:
#C_train = train.drop(['tripid','pickup_time', 'drop_time', 'pickup_date', 'drop_date', 'label', 'pick_lat', 'pick_lon', 'drop_lat', 'drop_lon'], axis = 1)
#C_test = test.drop(['tripid','pickup_time', 'drop_time', 'pickup_date', 'drop_date', 'pick_lat', 'pick_lon', 'drop_lat', 'drop_lon'], axis = 1)

C_train = train.drop(['tripid','pickup_time', 'drop_time', 'pickup_date', 'drop_date', 'label'], axis = 1)
C_test = test.drop(['tripid','pickup_time', 'drop_time', 'pickup_date', 'drop_date'], axis = 1)

In [176]:
x_train = C_train.iloc[:,C_train.columns!='label_bool']
y_train = C_train['label_bool'].values
x_test = C_test

In [177]:
x_train.to_csv('temp_train.csv', index=False)
x_test.to_csv('temp_test.csv', index=False)


# <span style="color:#c54e5e">Classification using Neural Networks</span>

##### <span style="color:#4ec57a">In this method I'm trying to predict the correctnes value which will be between 0 and 1. Prediction will be evaluated based on the value >= 0.5 or not</span>

In [178]:
train_df, validation_df = train_test_split(C_train, test_size=0.10, random_state=1)

# Get labels
train_labels = train_df['label_bool'].values
validation_labels = validation_df['label_bool'].values
train_df = train_df.drop(['label_bool'], axis=1)
validation_df = validation_df.drop(['label_bool'], axis=1)

In [179]:
test_clean = C_test

scaler = preprocessing.MinMaxScaler()
train_df_scaled = scaler.fit_transform(train_df)
validation_df_scaled = scaler.transform(validation_df)
test_scaled = scaler.transform(test_clean)

In [180]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

from keras.layers import Dense
from keras.models import Sequential
from keras.utils import to_categorical
from keras import regularizers


from keras.layers.normalization import BatchNormalization
from keras.optimizers import adam

In [181]:
# Model parameters
BATCH_SIZE = 256
LEARNING_RATE = 0.001
EPOCHS = 30

model = Sequential()
model.add(Dense(40, activation='relu', input_dim=train_df_scaled.shape[1], activity_regularizer=regularizers.l1(0.01)))
model.add(BatchNormalization())
model.add(Dense(50, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(25, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1))

adam = keras.optimizers.Adam(lr=LEARNING_RATE)  #keras.optimizers.adam(lr=LEARNING_RATE)
model.compile(loss='mse', optimizer="adam", metrics=['mae'])

In [182]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 40)                880       
_________________________________________________________________
batch_normalization_9 (Batch (None, 40)                160       
_________________________________________________________________
dense_14 (Dense)             (None, 50)                2050      
_________________________________________________________________
batch_normalization_10 (Batc (None, 50)                200       
_________________________________________________________________
dense_15 (Dense)             (None, 25)                1275      
_________________________________________________________________
batch_normalization_11 (Batc (None, 25)                100       
_________________________________________________________________
dense_16 (Dense)             (None, 1)                

In [183]:
history = model.fit(x=train_df_scaled, y=train_labels, batch_size=BATCH_SIZE, epochs=EPOCHS, 
                    verbose=1, validation_data=(validation_df_scaled, validation_labels), 
                    shuffle=True)

Train on 15246 samples, validate on 1695 samples
Epoch 1/30
15246/15246 [==============================] - 1s 96us/step - loss: 6.2963 - mae: 0.8161 - val_loss: 1.4068 - val_mae: 0.4919
Epoch 2/30
15246/15246 [==============================] - 0s 16us/step - loss: 0.8592 - mae: 0.5024 - val_loss: 0.2890 - val_mae: 0.2565
Epoch 3/30
15246/15246 [==============================] - 0s 17us/step - loss: 0.2882 - mae: 0.3142 - val_loss: 0.1500 - val_mae: 0.1392
Epoch 4/30
15246/15246 [==============================] - 0s 14us/step - loss: 0.1771 - mae: 0.2290 - val_loss: 0.1187 - val_mae: 0.0928
Epoch 5/30
15246/15246 [==============================] - 0s 14us/step - loss: 0.1382 - mae: 0.1968 - val_loss: 0.1039 - val_mae: 0.1041
Epoch 6/30
15246/15246 [==============================] - 0s 13us/step - loss: 0.1226 - mae: 0.1853 - val_loss: 0.0950 - val_mae: 0.1558
Epoch 7/30
15246/15246 [==============================] - 0s 14us/step - loss: 0.1165 - mae: 0.1837 - val_loss: 0.0917 - val_mae:

In [184]:
prediction = model.predict(test_scaled, batch_size=128, verbose=1)


8576/8576 [==============================] - 0s 47us/step


In [186]:
type(prediction) 

numpy.ndarray

In [187]:
sample_submission_nn = test.iloc[:, lambda df: [0]]
sample_submission_nn['label_bool'] = prediction

sample_submission_nn['prediction']=sample_submission_nn['label_bool'].apply(lambda x:1 if x >= 0.5 else 0)

sample_submission_nn = sample_submission_nn.drop(['label_bool'], axis = 1)
sample_submission_nn.to_csv('sample_submission_nn3.csv', index=False)

sample_submission_nn.head(5)

,tripid,prediction
0,213284604,1
1,213286352,1
2,213293973,1
3,213294622,1
4,213298687,1
